<a href="https://colab.research.google.com/github/jsansao/teic-20231/blob/main/TEIC_Licao31_bleu_bertscore.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 🤖 Comparando Métricas: BLEU vs. BERTScore

Este notebook demonstra a diferença fundamental entre a métrica **BLEU** (baseada em *n-grams*) e a métrica **BERTScore** (baseada em *semântica*).

Vamos avaliar 4 sentenças "candidatas" contra 1 sentença de "referência" para ver como cada métrica reage.

### O Cenário
* **BLEU:** Esperamos que pontue bem apenas quando as palavras *exatas* são usadas.
* **BERTScore:** Esperamos que pontue bem quando o *significado* é semelhante, mesmo que as palavras sejam diferentes.

In [6]:
# @title 1. Instalação das Bibliotecas
# Vamos instalar a biblioteca bert-score e a nltk (para o BLEU)

!pip install bert-score
!pip install nltk

In [7]:
# @title 2. Importações
import nltk
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
from bert_score import score
import pandas as pd

# Usaremos uma função de suavização para o BLEU,
# pois ele não funciona bem em sentenças curtas sem isso.
chen_smoothing = SmoothingFunction().method4

## 3. Definição das Sentenças de Teste

Este é o nosso caso de teste. Temos uma referência (a tradução "correta") e quatro candidatos com características diferentes:

1.  **Candidato 1 (Correspondência Exata):** Idêntico à referência.
2.  **Candidato 2 (Paráfrase/Semântico):** Significado idêntico, mas com sinônimos.
3.  **Candidato 3 (Sobreposição, Sentido Oposto):** Usa as mesmas palavras, mas o significado é o oposto.
4.  **Candidato 4 (Diferente):** Uma sentença completamente aleatória.

In [8]:
# @title 4. Nossas sentenças (em Português)

# A sentença que consideramos "correta"
referencia = "O gato rápido pulou sobre o cachorro preguiçoso."

# Lista de sentenças geradas para comparar
candidatos = [
    # 1. Correspondência Exata
    "O gato rápido pulou sobre o cachorro preguiçoso.",

    # 2. Paráfrase (Sinônimos, mesmo significado)
    "O felino ágil saltou por cima do cão sonolento.",

    # 3. Sobreposição de palavras (Significado oposto)
    "O cachorro preguiçoso pulou sobre o gato rápido.",

    # 4. Totalmente diferente
    "O céu está azul hoje."
]

# Vamos tokenizar (dividir as palavras) para o BLEU
ref_tokenized = [referencia.split()] # BLEU espera uma lista de referências
cands_tokenized = [c.split() for c in candidatos]

# BERTScore usa as sentenças brutas (strings)
# Precisamos de uma lista de referências com o mesmo tamanho da lista de candidatos
refs_bert = [referencia] * len(candidatos)

In [9]:
# @title 5. Cálculo das Métricas

# ----- 5.1. Calcular o BLEU -----
# BLEU é uma métrica a nível de corpus, mas podemos usá-la
# a nível de sentença com suavização.
scores_bleu = []
for cand in cands_tokenized:
    score_b = sentence_bleu(ref_tokenized, cand, smoothing_function=chen_smoothing)
    scores_bleu.append(score_b)


# ----- 5.2. Calcular o BERTScore -----
# Usaremos um modelo BERT treinado para o português para melhores resultados
# lang="pt" automaticamente usa "neuralmind/bert-base-portuguese-cased"
P, R, F1_bert = score(candidatos, refs_bert, lang="pt", verbose=False)

# Vamos extrair apenas o F1-score (a métrica principal)
scores_bert = F1_bert.tolist()

## 6. Análise dos Resultados

Vamos colocar tudo em uma tabela para comparar lado a lado.

* **Score_BLEU:** Varia de 0.0 a 1.0. Foca na sobreposição de palavras.
* **Score_BERT (F1):** Varia (aprox.) de 0.0 a 1.0. Foca na similaridade de significado.

In [10]:
# @title 7. Tabela Comparativa

data = {
    "Candidato": [
        "1. Exato",
        "2. Paráfrase (Sinônimos)",
        "3. Oposto (Sobreposição)",
        "4. Diferente"
    ],
    "Sentença": candidatos,
    "Score_BLEU": scores_bleu,
    "Score_BERT (F1)": scores_bert
}

df = pd.DataFrame(data)

# Vamos formatar os números para melhor leitura
df["Score_BLEU"] = df["Score_BLEU"].round(4)
df["Score_BERT (F1)"] = df["Score_BERT (F1)"].round(4)

print("Referência:", referencia)
print("-" * 50)
print(df.to_markdown(index=False))

Referência: O gato rápido pulou sobre o cachorro preguiçoso.
--------------------------------------------------
| Candidato                | Sentença                                         |   Score_BLEU |   Score_BERT (F1) |
|:-------------------------|:-------------------------------------------------|-------------:|------------------:|
| 1. Exato                 | O gato rápido pulou sobre o cachorro preguiçoso. |       1      |            1      |
| 2. Paráfrase (Sinônimos) | O felino ágil saltou por cima do cão sonolento.  |       0.0257 |            0.836  |
| 3. Oposto (Sobreposição) | O cachorro preguiçoso pulou sobre o gato rápido. |       0.1963 |            0.9605 |
| 4. Diferente             | O céu está azul hoje.                            |       0.0251 |            0.6774 |


## 8. Conclusão da Análise

Veja o que a tabela nos mostra:

1.  **Candidato 1 (Exato):**
    * **BLEU:** 1.00 (Perfeito).
    * **BERTScore:** 1.00 (Perfeito).
    * *Ambas as métricas concordam que uma correspondência exata é perfeita.*

2.  **Candidato 2 (Paráfrase/Sinônimos):**
    * **BLEU:** ~0.0 (Muito baixo). Não há quase nenhuma palavra em comum (apenas "O" e "do").
    * **BERTScore:** ~0.84 (Muito alto). O BERT entende que "gato" é sinônimo de "felino", "rápido" de "ágil", "pulou" de "saltou", etc.
    * *Esta é a maior falha do BLEU e a maior força do BERTScore.*

3.  **Candidato 3 (Oposto):**
    * **BLEU:** ~0.20 (Médio-Alto). O BLEU vê que *todas* as palavras corretas estão presentes, ele só não entende a ordem e, portanto, a mudança de significado (quem pulou em quem).
    * **BERTScore:** ~0.96 (Alto). O BERTScore também é "enganado" pela alta sobreposição de palavras, mas geralmente dá uma pontuação ligeiramente menor que uma paráfrase correta, pois o contexto dos embeddings muda.
    * *(Nota: A robustez do BERTScore a mudanças de ordem pode variar com o modelo).*

4.  **Candidato 4 (Diferente):**
    * **BLEU:** ~0.0 (Muito baixo).
    * **BERTScore:** ~0.67 (Baixo).
    * *Ambas as métricas concordam que esta é uma péssima correspondência, mas o BERTScore raramente dá 0.0, pois sempre há alguma similaridade semântica residual (palavras como "O", "está", etc.).
  